In [5]:
# example of defining the discriminator model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.utils import plot_model
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
 model = Sequential()
 model.add(Conv2D(64, (3,3), strides=(2, 2), padding="same", input_shape=in_shape))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Dropout(0.4))
 model.add(Conv2D(64, (3,3), strides=(2, 2), padding="same"))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Dropout(0.4))
 model.add(Flatten())
 model.add(Dense(1, activation="sigmoid"))
# compile model
 opt = Adam(lr=0.0002, beta_1=0.5)
 model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
 return model
# define model
model = define_discriminator()
# summarize the model
model.summary()
# plot the model
plot_model(model, to_file="discriminator_plot.png", show_shapes=True, show_layer_names=True)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 14, 14, 64)        640       
_________________________________________________________________
leaky_re_lu (LeakyReLU)      (None, 14, 14, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 7, 7, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 7, 7, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 3136)              0

In [8]:
# load and prepare mnist training images
def load_real_samples():
# load mnist dataset
 (trainX, _), (_, _) = load_data()
# expand to 3d, e.g. add channels dimension
 X = expand_dims(trainX, axis=-1)
# convert from unsigned ints to floats
 X = X.astype("float32")
# scale from [0,255] to [0,1]
 X = X / 255.0
 return X

In [9]:
# select real samples
def generate_real_samples(dataset, n_samples):
# choose random instances
 ix = randint(0, dataset.shape[0], n_samples)
# retrieve selected images
 X = dataset[ix]
# generate ✬real✬ class labels (1)
 y = ones((n_samples, 1))
 return X, y

In [11]:
# generate n fake samples with class labels
def generate_fake_samples(n_samples):
# generate uniform random numbers in [0,1]
 X = rand(28 * 28 * n_samples)
# reshape into a batch of grayscale images
 X = X.reshape((n_samples, 28, 28, 1))
# generate ✬fake✬ class labels (0)
 y = zeros((n_samples, 1))
 return X, y

In [5]:
# example of training the discriminator model on real and random mnist images
import tensorflow as tf
from numpy import expand_dims
from numpy import ones
from numpy import zeros
from numpy.random import rand
from numpy.random import randint
from tensorflow.keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
 model = Sequential()
 model.add(Conv2D(64, (3,3), strides=(2, 2), padding="same", input_shape=in_shape))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Dropout(0.4))
 model.add(Conv2D(64, (3,3), strides=(2, 2), padding="same"))
 model.add(LeakyReLU(alpha=0.2))
 model.add(Dropout(0.4))
 model.add(Flatten())
 model.add(Dense(1, activation="sigmoid"))
# compile model
 opt = Adam(lr=0.0002, beta_1=0.5)
 model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])
 return model
# load and prepare mnist training images
def load_real_samples():
# load mnist dataset
 (trainX, _), (_, _) = load_data()
# expand to 3d, e.g. add channels dimension
 X = expand_dims(trainX, axis=-1)
# convert from unsigned ints to floats
 X = X.astype("float32")
# scale from [0,255] to [0,1]
 X = X / 255.0
 return X
# select real samples
def generate_real_samples(dataset, n_samples):
# choose random instances
 ix = randint(0, dataset.shape[0], n_samples)
# retrieve selected images
 X = dataset[ix]
# generate ✬real✬ class labels (1)
 y = ones((n_samples, 1))
 return X, y
# generate n fake samples with class labels
def generate_fake_samples(n_samples):
# generate uniform random numbers in [0,1]
 X = rand(28 * 28 * n_samples)
# reshape into a batch of grayscale images
 X = X.reshape((n_samples, 28, 28, 1))
# generate ✬fake✬ class labels (0)
 y = zeros((n_samples, 1))
 return X, y
# train the discriminator model
def train_discriminator(model, dataset, n_iter=100, n_batch=256):
 half_batch = int(n_batch / 2)
# manually enumerate epochs
 for i in range(n_iter):
# get randomly selected ✬real✬ samples
  X_real, y_real = generate_real_samples(dataset, half_batch)
# update discriminator on real samples
  _, real_acc = model.train_on_batch(X_real, y_real)
# generate ✬fake✬ examples
  X_fake, y_fake = generate_fake_samples(half_batch)
# update discriminator on fake samples
  _, fake_acc = model.train_on_batch(X_fake, y_fake)
# summarize performance
  print('>%d real=%.0f%% fake=%.0f%%' % (i+1, real_acc*100, fake_acc*100))
# define the discriminator model
model = define_discriminator()
# load image data
dataset = load_real_samples()
# fit the model
train_discriminator(model, dataset)

>1 real=46% fake=52%
>2 real=52% fake=75%
>3 real=59% fake=82%
>4 real=55% fake=96%
>5 real=58% fake=98%
>6 real=61% fake=99%
>7 real=55% fake=100%
>8 real=59% fake=100%
>9 real=59% fake=100%
>10 real=65% fake=100%
>11 real=71% fake=100%
>12 real=66% fake=100%
>13 real=71% fake=100%
>14 real=72% fake=100%
>15 real=72% fake=100%
>16 real=74% fake=100%
>17 real=75% fake=100%
>18 real=77% fake=100%
>19 real=84% fake=100%
>20 real=86% fake=100%
>21 real=83% fake=100%
>22 real=95% fake=100%
>23 real=93% fake=100%
>24 real=93% fake=100%
>25 real=90% fake=100%
>26 real=94% fake=100%
>27 real=95% fake=100%
>28 real=98% fake=100%
>29 real=98% fake=100%
>30 real=98% fake=100%
>31 real=98% fake=100%
>32 real=98% fake=100%
>33 real=98% fake=100%
>34 real=99% fake=100%
>35 real=100% fake=100%
>36 real=100% fake=100%
>37 real=99% fake=100%
>38 real=100% fake=100%
>39 real=99% fake=100%
>40 real=99% fake=100%
>41 real=100% fake=100%
>42 real=100% fake=100%
>43 real=100% fake=100%
>44 real=99% fake=10